In [ ]:
import mesa


class MyAgent(mesa.Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name

    def step(self):
        print("{} activated".format(self.name))
        # Whatever else the agent does when activated


class MyModel(mesa.Model):
    def __init__(self, n_agents):
        super().__init__()
        self.schedule = mesa.time.RandomActivation(self)
        self.grid = mesa.space.MultiGrid(10, 10, torus=True)
        self.datacollector = mesa.DataCollector(model_reporters={"agent_count":
                                                                 lambda m: m.schedule.get_agent_count()},
                                                agent_reporters={"name": lambda a: a.name})
        for i in range(n_agents):
            a = MyAgent(i, self)
            self.schedule.add(a)
            coords = (self.random.randrange(0, 10),
                      self.random.randrange(0, 10))
            self.grid.place_agent(a, coords)

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

In [ ]:
model = MyModel(5)
for t in range(10):
    model.step()
model_df = model.datacollector.get_model_vars_dataframe()
agent_df = model.datacollector.get_agent_vars_dataframe()

In [ ]:
model_df

In [ ]:
agent_df

In [ ]:
import mesa

parameters = {"n_agents": range(1, 20)}
batch_run = mesa.batch_run(
    MyModel,
    parameters,
    max_steps=10,
)

In [ ]:
batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
import mesa

def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "Color": "red",
                 "r": 0.5}
    return portrayal

grid = mesa.visualization.CanvasGrid(agent_portrayal, 10, 10, 500, 500)
server = mesa.visualization.ModularServer(MyModel,
                       [grid],
                       "My Model",
                       {'n_agents': 10})
server.launch()